In [3]:
import pandas as pd
from datetime import datetime, timedelta
from iexfinance.stocks import get_historical_data
import iexfinance as iex
from iexfinance.refdata import get_symbols

In [4]:
# Use the `get_symbols` function from the iexfinance SDK to fetch available stock tickers
tickers=pd.DataFrame(get_symbols())
tickers.head()

,currency,date,exchange,iexId,isEnabled,name,region,symbol,type
0,USD,2019-09-18,NYS,IEX_46574843354B2D52,True,Agilent Technologies Inc.,US,A,cs
1,USD,2019-09-18,NYS,IEX_4238333734532D52,True,Alcoa Corp.,US,AA,cs
2,USD,2019-09-18,PSE,IEX_474B433136332D52,True,Perth Mint Physical Gold ETF,US,AAAU,et
3,USD,2019-09-18,NAS,IEX_4E5434354A302D52,True,Altaba Inc.,US,AABA,cef
4,USD,2019-09-18,NYS,IEX_4843364642592D52,True,AAC Holdings Inc.,US,AAC,cs


In [13]:
list_of_tickers = ["ACB", "GWPH", "CRON", "CGC", "TLRY", "APHA"]
# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-365)

# Get 1 year's worth of historical data for "SPY" and "AGG"
df = get_historical_data(list_of_tickers, start_date, end_date,  output_format='pandas')
df.head()

ACB                          GWPH                          \
           open high low close volume    open    high     low   close   
date                                                                    
2018-09-17  NaN  NaN NaN   NaN    NaN  141.31  146.89  140.50  140.94   
2018-09-18  NaN  NaN NaN   NaN    NaN  141.54  151.71  140.38  149.20   
2018-09-19  NaN  NaN NaN   NaN    NaN  154.00  166.42  150.10  153.64   
2018-09-20  NaN  NaN NaN   NaN    NaN  155.03  174.50  154.47  170.10   
2018-09-21  NaN  NaN NaN   NaN    NaN  170.01  172.50  163.00  164.98   

                     ...    TLRY                                     APHA  \
             volume  ...    open    high     low   close    volume   open   
date                 ...                                                    
2018-09-17   250925  ...  117.14  126.10  114.00  120.19  11682962  15.53   
2018-09-18   713691  ...  134.65  155.10  132.00  154.98  19955697  15.66   
2018-09-19  2464533  ...  233.58  300.00  151.40  214.06  31717724  16.37   
2018-09-20  2913383  ...  214.11  244.00  158.00  176.35  20493545  15.31   
2018-09-21   975812  ...  148.87  148.88  121.51  123.00  16097705  15.13   

                                          
             high    low  close   volume  
date                                      
2018-09-17  15.71  14.49  15.00  2227103  
2018-09-18  16.17  15.22  16.00  2750233  
2018-09-19  16.66  14.33  14.99  4916190  
2018-09-20  15.85  14.43  15.45  2594738  
2018-09-21  15.40  14.35  15.31  1646443  

[5 rows x 30 columns]

In [14]:
## Cleaning the Data
df.drop(columns=['open', 'high', 'low', 'volume'], level=1, inplace=True)
df.head()

,ACB,GWPH,CRON,CGC,TLRY,APHA
,close,close,close,close,close,close
date,,,,,,
2018-09-17,NaN,140.94,10.18,48.60,120.19,15.00
2018-09-18,NaN,149.20,11.56,51.89,154.98,16.00
2018-09-19,NaN,153.64,12.69,49.35,214.06,14.99
2018-09-20,NaN,170.10,13.75,52.40,176.35,15.45
2018-09-21,NaN,164.98,12.51,49.78,123.00,15.31
